# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
#!unzip data.zip
#!ls 

Permission denied: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
Maybe you need to change permission over 'Anyone with the link'?
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/ML\ NTU

/content/drive/MyDrive/ML NTU


In [ ]:
!ls

 model.ckpt	        test_11.npy	  train_11.npy.zip
'pred_Adam BN256.npy'   test_11.npy.zip   train_label_11.npy
 prediction.csv         train_11.npy	  train_label_11.npy.zip


In [ ]:
!unzip train_label_11.npy.zip
!unzip train_11.npy.zip
!unzip test_11.npy.zip

Archive:  train_label_11.npy.zip
replace train_label_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  train_11.npy.zip
replace train_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test_11.npy.zip
replace test_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

#data_root='timit_11/'
train = np.load('train_11.npy')
train_label = np.load('train_label_11.npy')
test = np.load('test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1229932, 429)
Size of validation set: (0, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 256

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

147

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [2]:
import torch
import torch.nn as nn

class Classifier1(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1800)
        self.layer2 = nn.Linear(1800, 1000)
        self.layer3 = nn.Linear(1000, 450)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(450, 39) 
        self.bn1 = nn.BatchNorm1d(1800)
        self.bn1 = nn.BatchNorm1d(1000)
        self.bn1 = nn.BatchNorm1d(450)

        self.act_fn_1 = nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn_1(x)

        x = self.out(x)
        
        
        return x

In [3]:
class Classifier2(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1600)
        self.layer2 = nn.Linear(1600, 1000)
        self.layer3 = nn.Linear(1000, 500)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(500, 39) 
        self.bn1 = nn.BatchNorm1d(1600)
        self.bn1 = nn.BatchNorm1d(1000)
        self.bn1 = nn.BatchNorm1d(500)

        self.act_fn_1 = nn.ReLU()
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn_1(x)

        x = self.out(x)
        
        
        return x

In [4]:
class Classifier3(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1200)
        self.layer2 = nn.Linear(1200, 900)
        self.layer3 = nn.Linear(900, 500)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(500, 39) 
        self.bn1 = nn.BatchNorm1d(1200)
        self.bn1 = nn.BatchNorm1d(900)
        self.bn1 = nn.BatchNorm1d(500)

        self.act_fn_1 = nn.ReLU()
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn_1(x)

        x = self.out(x)
        
        
        return x

In [ ]:
class Classifier4(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1500)
        self.layer2 = nn.Linear(1500, 1000)
        self.layer3 = nn.Linear(1000, 400)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(400, 39) 


        self.act_fn_1 = nn.ReLU()
    def forward(self, x):
        x = self.layer1(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.act_fn_1(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.act_fn_1(x)

        x = self.out(x)
        
        
        return x

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 200               # number of training epoch
learning_rate = 0.0001      # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier1().to(device)
mode2 = Classifier2().to(device)
mode3 = Classifier3().to(device)
mode4 = Classifier4().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [ ]:
# start training

#分別對model1 2 3 4 用全部data 訓練 200 ephoch
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model1(inputs) 
        #outputs = model2(inputs)
        #outputs = model3(inputs)
        #outputs = model4(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model1.eval() 
        #model2.eval()
        #model3.eval()
        #model4.eval()  # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model1(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model1.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model1.state_dict(), model_path)
    #torch.save(model2.state_dict(), model_path)
    #torch.save(model3.state_dict(), model_path)
    #torch.save(model4.state_dict(), model_path)
    print('saving model at last epoch')


[001/200] Train Acc: 0.428850 Loss: 1.933589
[002/200] Train Acc: 0.527768 Loss: 1.541844
[003/200] Train Acc: 0.566109 Loss: 1.402606
[004/200] Train Acc: 0.591413 Loss: 1.311791
[005/200] Train Acc: 0.608012 Loss: 1.250058
[006/200] Train Acc: 0.620625 Loss: 1.204960
[007/200] Train Acc: 0.630392 Loss: 1.169930
[008/200] Train Acc: 0.638599 Loss: 1.140769
[009/200] Train Acc: 0.645270 Loss: 1.116353
[010/200] Train Acc: 0.651256 Loss: 1.095270
[011/200] Train Acc: 0.656096 Loss: 1.077485
[012/200] Train Acc: 0.660314 Loss: 1.061820
[013/200] Train Acc: 0.664571 Loss: 1.047175
[014/200] Train Acc: 0.668437 Loss: 1.034284
[015/200] Train Acc: 0.671302 Loss: 1.022837
[016/200] Train Acc: 0.673965 Loss: 1.012222
[017/200] Train Acc: 0.676204 Loss: 1.003088
[018/200] Train Acc: 0.679154 Loss: 0.994127
[019/200] Train Acc: 0.681253 Loss: 0.986070
[020/200] Train Acc: 0.683640 Loss: 0.978582
[021/200] Train Acc: 0.685290 Loss: 0.970809
[022/200] Train Acc: 0.687384 Loss: 0.963903
[023/200] 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
predict_class = []
predict_possible = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict_class.append(y)
        for x in outputs:
            predict_possible.append(x)

In [ ]:
predict_1 = predict_possible
#predict_2 = predict_possible
#predict_3 = predict_possible
#predict_4 = predict_possible

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
# 直接相加最後 output 機率 model表現較好用 1.1權重
class_mix = []
possibility_mix = []
for i in range(len(predict_possible)):
    mix =  predict_1[i] + predict_2[i] + 1.1 * predict_3[i] + predict_4[i]
    mix = mix.cpu().numpy()
    possibility_mix.append(mix)
    label = np.argmax(mix)
    class_mix.append(label)

In [ ]:
predict_class = class_mix
predict_possible = possibility_mix

In [ ]:
#post process
predict_class_new=[]
for i in range(len(predict_class)):
    if (i==0) or (i == len(predict_class)-1):
        predict_class_new.append(predict_class[i])
        continue
    else:
        if predict_class[i] == predict_class[i-1] or predict_class[i] == predict_class[i+1]:
            predict_class_new.append(predict_class[i])
        else:
          possible = predict_possible[i]
          possible_sort = np.sort(possible)
          for order in range(-2,-4,-1):
            order_max = possible_sort[order]
            index = np.where(possible == order_max)[0][0]
            if index == predict_class[i-1] or index == predict_class[i+1]:
                print(i)
                predict_class_new.append(index)
                break
          else:
            predict_class_new.append(predict_class[i])

串流輸出內容已截斷至最後 5000 行。
247045
247358
247383
247384
247385
247517
247651
247652
247653
247666
247667
247682
247733
247738
247739
247976
247977
247994
248004
248005
248063
248067
248086
248100
248124
248156
248172
248183
248187
248188
248189
248220
248240
248245
248324
248380
248457
248458
248480
248500
248506
248525
248552
248628
248689
248709
248710
248711
248762
248763
248766
248785
248804
248815
248825
248864
248869
248884
248938
248949
248957
248978
248981
248994
248995
249002
249003
249041
249059
249066
249067
249084
249114
249133
249134
249204
249415
249447
249462
249482
249497
249498
249533
249534
249553
249709
250117
250118
250210
250397
250398
250399
250400
250450
250464
250508
250524
250544
250545
250546
250547
250563
250685
250861
250926
251012
251047
251149
251227
251228
251229
251230
251375
251477
251538
251575
251633
251638
251645
251675
251680
251750
251830
251858
251861
251923
252066
252074
252142
252157
252163
252201
252262
252339
252379
252631
252672
252945
253007
253008

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict_class_new):
        f.write('{},{}\n'.format(i, y))